In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install nemo_toolkit['all']
!pip install hydra-core==1.1
!pip install import-ipynb

In [3]:
%cd "/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation"

/content/drive/MyDrive/Colab Notebooks/Paper 1 Implementation


In [ ]:
import torch
import torch.nn as nn

import import_ipynb
%run rbkd.ipynb import RBKDLoss
%run ebkd.ipynb import EBKDLoss
from typing import Dict, Optional, Union
from omegaconf import DictConfig, ListConfig, OmegaConf, open_dict
from nemo.core.classes import Serialization, Typing, typecheck
from nemo.collections.asr.losses.ctc import CTCLoss
from nemo.utils import logging, model_utils
from nemo.core.neural_types import LossType, NeuralType, LogprobsType, LabelsType, LengthsType

In [5]:
_all_ = ['TotalLoss']

In [6]:
class TotalLoss(CTCLoss):

    @property
    def input_types(self):
        """Input types definitions for TotalLoss.
        """
        return {
            "teacher_logits": NeuralType(('B', 'T', 'D'), LogprobsType()),
            "log_probs": NeuralType(('B', 'T', 'D'), LogprobsType()),
            "targets": NeuralType(('B', 'T'), LabelsType()),
            "input_lengths": NeuralType(tuple('B'), LengthsType()),
            "target_lengths": NeuralType(tuple('B'), LengthsType()),
            "teacher_feature_map": NeuralType(('B', 'T', 'D'), LogprobsType()),
            "student_feature_map": NeuralType(('B', 'T', 'D'), LogprobsType())
        }

    @property
    def output_types(self):
        """Output types definitions for TotalLoss.
        loss:
            NeuralType(None)
        """
        return {"loss": NeuralType(elements_type=LossType())}

    def __init__(self, *args, **kwargs):

        # Calling Parent Class to access its methods 
        super().__init__(
            num_classes=kwargs.get("num_classes"),
            zero_infinity=True,
            reduction=kwargs.get("reduction", "mean_batch")
        )

        # Hyperparameters
        self.gamma = kwargs.get("gamma", 500)
        self.beta = kwargs.get("beta", 0.03)
        self.temperature = kwargs.get("temperature", 3)

        # CTC Loss Object Creation
        self.CTCLoss = CTCLoss(
            num_classes=kwargs.get("num_classes"),
            zero_infinity=True,
            reduction=kwargs.get("reduction", "mean_batch"),
        )

        # RBKD Loss Object Creation
        self.RBKDLoss = RBKDLoss(
            num_classes=kwargs.get("num_classes"),
            zero_infinity=True,
            reduction=kwargs.get("reduction", "mean_batch"),
            temperature=self.temperature
        )

        # EBKD Loss Object Creation
        self.EBKDLoss = EBKDLoss(
            zero_infinity=True,
            reduction=kwargs.get("reduction", "mean_batch"),
        )

    @typecheck
    def forward(self, log_probs, targets, input_lengths, target_lengths,
                teacher_logits, teacher_feature_map, student_feature_map):
        
        # Calling methods of CTC Loss 
        ctc_loss = self.CTCLoss(
            log_probs, targets, input_lengths, target_lengths
        )

        # Calling methods of RBKD Loss
        rbkd_loss = self.RBKDLoss(
            teacher_logits, log_probs
        )

        # Calling methods of EBKD Loss
        ebkd_loss = self.EBKDLoss(
            teacher_logits, log_probs, teacher_feature_map, student_feature_map
        )

        # Returning Total Loss
        return ctc_loss + (self.beta * rbkd_loss) + (self.gamma * ebkd_loss)